# 전체_도시철도역사정보_20250417.xlsx 파일 내 좌표 확인
## KRIC 레일 포털에서 도시철도에 관한 정보 검색
### KRIC 레일(https://data.kric.go.kr/rips/) -> 개방데이터 -> 파일데이터에서 역사정보 검색
- 표준데이터 역사정보(전체 기관)와 전국 도시광역철도 역사정보의 엑셀 파일을 다운받아 부산광역시에서 운행하는 노선의 데이터기준일자를 참고하여 최신화 여부 확인
    - 표준데이터 역사정보(전체 기관)
        - https://data.kric.go.kr/rips/M_01_01/detail.do?id=32&keywords=%ec%97%ad%ec%82%ac%ec%a0%95%eb%b3%b4&lcd=&mcd=
        - 전체_도시철도역사정보_20250417.xlsx 다운로드
    - 전국 도시광역철도 역사정보
        - https://data.kric.go.kr/rips/M_01_01/detail.do?id=1294&keywords=%ec%97%ad%ec%82%ac%ec%a0%95%eb%b3%b4&lcd=&mcd=
        - 전국 도시광역철도_역사정보_20250101.xlsx 다운로드

In [ ]:
import pandas as pd

df1 = pd.read_excel('../../Data_list/v1.0/전체_도시철도역사정보_20250417.xlsx', index_col='역번호')
# 전체_도시철도역사정보_20250417.xlsx 파일을 읽어서 DataFrame 생성, '역번호'를 인덱스로 사용

df1 = pd.DataFrame(df1, columns=['역사명', '노선명', '환승역구분', '환승노선명', '역사도로명주소', '역위도', '역경도', '데이터기준일자'])
# 필요한 컬럼만 추출하여 새로운 DataFrame 생성

df1.index = df1.index.astype(str).str.zfill(4)
# 인덱스(역번호)를 문자열로 변환하고 4자리로 맞춤(예: 1 → 0001)

df1 = df1.query("노선명 in ['부산 도시철도 1호선', '부산 도시철도 2호선', '부산 도시철도 3호선', '부산 경량도시철도 4호선', '부산김해경전철', '동해선']").sort_index()
# 부산 도시철도 1~4호선, 부산김해경전철, 동해선만 필터링하여 정렬

df2 = pd.read_excel('../../Data_list/v2.0/전국 도시광역철도_역사정보_20250101.xlsx')
# 전국 도시광역철도_역사정보_20250101.xlsx 파일을 읽어서 DataFrame 생성

df2 = pd.DataFrame(df2, columns=['철도운영기관명', '운영노선', '역명(한글)', '역 주소(도로명 주소)', '역 위치(위도)', '역 위치(경도)', '데이터 기준일자'])
# 필요한 컬럼만 추출하여 새로운 DataFrame 생성

df2 = df2.query("운영노선 in ['동해선'] or 철도운영기관명 in ['부산교통공사', '부산김해경전철']")
# 동해선 또는 부산교통공사(1~4호선이 포함됨), 부산김해경전철 운영노선만 필터링

df2 = df2.sort_values(by='운영노선', ascending=True)
# 운영노선 기준으로 오름차순 정렬

print(len(df1.index))
print(len(df2.index))
# 부산 도시철도 역 수가 158개가 맞는지 확인
# 출력 결과 모두 158로 정상적으로 출력됨

In [ ]:
pd.set_option('display.max_rows', None)  # DataFrame 출력 시 모든 행을 화면에 표시하도록 설정
df1 # 수집된 데이터를 출력하여 데이터기준일자 검토

In [ ]:
df2 # 수집된 데이터를 출력하여 데이터기준일자 검토

### 데이터기준일자 확인 결과
- df1 = 전체_도시철도역사정보_20250417.xlsx
    - 부산 도시철도 1호선부터 4호선까지: 2022-11-30 00:00:00
    - 부산김해경전철: 2022-04-27 00:00:00
    - 동해선: 2025-04-08 00:00:00
- df2 = 전국 도시광역철도_역사정보_20250101.xlsx
    - 1호선부터 4호선까지: 20241231
    - 부산김해선: 20250106
    - 동해선: 20230509

- 부산 도시철도 1~4호선과 부산김해선은 전국 도시광역철도_역사정보_20250101.xlsx가 더 최신 버전
- 동해선은 전체_도시철도역사정보_20250417.xlsx가 더 최신 버전
- 아래 코드를 실행하여 각 파일별로 더 최신 버전인 노선만 분류하고 CSV 파일로 저장

In [ ]:
df1.drop(df1[df1['노선명'] != '동해선'].index, inplace=True)
# df1에서 '노선명'이 '동해선'이 아닌 행을 모두 삭제하여 동해선 데이터만 남김

df2.drop(df2[df2['운영노선'] == '동해선'].index, inplace=True)
# df2에서 '운영노선'이 '동해선'인 행을 모두 삭제하여 동해선을 제외한 데이터만 남김

df1.to_csv('부산광역시 도시철도 동해선.csv')
# 동해선 데이터만 남은 df1을 '부산광역시 도시철도 동해선.csv' 파일로 저장

df2.to_csv('부산광역시 도시철도 1~4호선, 부산김해경전철.csv')
# 동해선을 제외한 df2를 '부산광역시 도시철도 1~4호선, 부산김해경전철.csv' 파일로 저장

### QGIS로 두 csv 파일을 레이어로 추가한 결과 부산광역시 도시철도 동해선의 역 위치가 QGIS의 QuickMapServices 플러그인으로 확인하는 지도의 동해선과 노선 위치가 차이나는 역이 많았습니다. 아래 번호 순서대로 차이점을 확인할 수 있도록 동해선.pdf 파일을 첨부합니다.

1. 태화강 ~ 덕하역 위치가 노선에서 약간 벗어남
2. 망양역 ~ 남창역 위치가 노선에서 약간 벗어남
3. 서생역 ~ 좌천역
4. 벡스코역
5. 부산원동역의 위치가 부자연스러움
6. 교대역 ~ 거제역
7. 송정역의 위치가 부산광역시 송정역이 아닌 광주광역시의 광주송정역으로 표시됨

따라서 전체_도시철도역사정보_20250417.xlsx 파일의 동해선 역위도와 역경도가 전체적으로 부정확하다고 판단하고 역사도로명주소 컬럼을 이용하여 좌표를 새로 변환하기로 했습니다.

## 주소를 좌표로 변환하는 오픈API 이용
### 브이월드의 Geocoder API 2.0 레퍼런스 활용
- https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do

In [ ]:
import pandas as pd
import requests
import time
import random

all_addresses = df1['역사도로명주소'].dropna().tolist()
# '역사도로명주소' 컬럼의 모든 주소를 리스트로 추출

address_coord = {}
# 주소별 좌표값을 저장하기 위한 빈 딕셔너리 생성

for addr in all_addresses:
    apiurl = 'https://api.vworld.kr/req/address?'
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": addr,
        "format": "json",
        "type": "parcel", # 지번 주소로 좌표 변환 시도
        "key": '3BDEB80E-7485-3E95-9639-1E81D0E02869', # 발급받은 API 키를 작은따옴표 사이에 입력
    }
    response = requests.get(apiurl, params=params)
    time.sleep(random.uniform(1, 2))  # 1~2초 대기(과도한 요청으로 인한 서버 부담 방지)
    if response.status_code == 200:
        response_json = response.json()
        if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
            point = response_json['response']['result']['point']
            # 정상 응답이면서 좌표(point) 정보가 있으면 아래 코드 실행

            address_coord[addr] = (point['y'], point['x'])  # 위도, 경도 저장    
            print(f"주소 '{addr}' 지번 변환 성공: 위도 {point['y']}, 경도 {point['x']}")
            # 위도/경도 출력 및 저장이 성공했음을 확인하기 쉽도록 메시지 출력
        
        else:
            # 지번 주소로 실패 시 도로명 주소로 재시도
            print(f"주소 '{addr}' 지번 변환 실패. 재시도")
            params['type'] = 'road'
            response = requests.get(apiurl, params=params)
            time.sleep(random.uniform(1, 2))
            if response.status_code == 200:
                response_json = response.json()
                if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
                    point = response_json['response']['result']['point']
                    address_coord[addr] = (point['y'], point['x'])
                    print(f"주소 '{addr}' 도로명 변환 성공: 위도 {point['y']}, 경도 {point['x']}")
                else:
                    # 지번/도로명 주소 모두 실패하면 아래 메시지 출력
                    print(f"주소 '{addr}' 도로명 변환 실패")

            else: # API 요청 자체가 실패한 경우 아래 메시지 출력
                print(f"주소 '{addr}' 도로명 변환 요청 실패: 상태 코드 {response.status_code}")
    else:
        print(f"주소 '{addr}' 지번 변환 요청 실패: 상태 코드 {response.status_code}")

# 변환된 좌표를 df1의 해당 주소 행에 반영
for addr, (lat, lon) in address_coord.items():
    df1.loc[df1['역사도로명주소'] == addr, '역위도'] = lat
    df1.loc[df1['역사도로명주소'] == addr, '역경도'] = lon

# 수정한 데이터프레임을 부산광역시 도시철도 동해선.csv에 덮어쓰기
df1.to_csv('부산광역시 도시철도 동해선.csv')

### 출력 결과에서 좌표 변환에 실패한 주소 확인

주소 '부산광역시 기장군 일광면 이천리 750-4' 지번 변환 실패. 재시도

주소 '부산광역시 기장군 일광면 이천리 750-4' 도로명 변환 실패


주소 '울산시 남구 상개리 97-1' 지번 변환 실패. 재시도

주소 '울산시 남구 상개리 97-1' 도로명 변환 실패



1. 부산광역시 기장군 일광면 이천리 750-4
    - 주민등록업무 담당 행정기관 및 관할구역 변경내역(2022. 4. 1. 시행) 확인 결과 일광면을 폐지하고 일광읍을 신설했습니다.
    - 따라서 주소를 부산광역시 기장군 일광읍 이천리 750-4로 변경하고 다시 좌표로 변환했습니다.
    - 출처: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=91305
2. 울산시 남구 상개리 97-1
    - 1997년 7월 울산시가 광역시로 승격하면서 울산광역시 남구 상개동으로 편제됐음을 확인했습니다.
    - 따라서 주소를 울산광역시 남구 상개동 97-1로 변경하고 다시 좌표로 변환했습니다.
    - 출처: https://ulsan.grandculture.net/Contents/Index?local=ulsan&gcode=02&contents_id=GC80020091

아래 코드를 실행하여 다음을 수행
1. 수정 전 주소를 수정한 주소로 갱신
2. 기존 좌표를 수정한 주소로 변환한 좌표로 갱신

In [ ]:
import pandas as pd
import requests
import time
import random

pd.set_option('display.max_rows', None)
address1 = ['부산광역시 기장군 일광면 이천리 750-4', '울산시 남구 상개리 97-1']
# 수정 전 주소 목록 (좌표 변환이 실패했던 원본 주소)
address2 = ['부산광역시 기장군 일광읍 이천리 750-4', '울산광역시 남구 상개동 97-1']
# 수정 후 주소 목록

for old_addr, new_addr in zip(address1, address2):
    # address1과 address2를 동시에 순회하며
    # old_addr(수정 전 주소), new_addr(수정 후 주소)로 사용
    apiurl = 'https://api.vworld.kr/req/address?'
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": new_addr, # 수정된 주소로 좌표 변환 요청
        "format": "json",
        "type": "parcel",
        "key": '3BDEB80E-7485-3E95-9639-1E81D0E02869',
    }
    response = requests.get(apiurl, params=params)
    time.sleep(random.uniform(1, 2))
    if response.status_code == 200:
        response_json = response.json()
        if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
            point = response_json['response']['result']['point']
            # 정상 응답이면 좌표(point) 정보 추출

            df1.loc[df1['역사도로명주소'] == old_addr, '역위도'] = point['y']
            df1.loc[df1['역사도로명주소'] == old_addr, '역경도'] = point['x']
            # df1에서 old_addr(수정 전 주소)에 해당하는 행의 역위도/역경도 값을 새 좌표로 수정
            
            df1.loc[df1['역사도로명주소'] == old_addr, '역사도로명주소'] = new_addr
            # 그리고 주소 컬럼도 old_addr에서 new_addr로 수정
            print(f"주소 '{old_addr}' → '{new_addr}' 변경 완료 및 좌표 수정 완료: 역위도 {point['y']}, 역경도 {point['x']}")
        else:
            # 지번 주소로 실패 시 도로명 주소로 재시도
            print(f"주소 '{new_addr}' 좌표 변환 실패. 재시도")
            params['type'] = 'road'
            response = requests.get(apiurl, params=params)
            time.sleep(random.uniform(1, 2))
            if response.status_code == 200:
                response_json = response.json()
                if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
                    point = response_json['response']['result']['point']
                    # 정상 응답이면 좌표(point) 정보 추출

                    df1.loc[df1['역사도로명주소'] == old_addr, '역위도'] = point['y']
                    df1.loc[df1['역사도로명주소'] == old_addr, '역경도'] = point['x']
                    # df1에서 old_addr(수정 전 주소)에 해당하는 행의 역위도/역경도 값을 새 좌표로 수정
                    
                    df1.loc[df1['역사도로명주소'] == old_addr, '역사도로명주소'] = new_addr
                    # 그리고 주소 컬럼도 old_addr에서 new_addr로 수정
                    print(f"주소 '{old_addr}' → '{new_addr}' 변경 완료 및 좌표 수정 완료: 역위도 {point['y']}, 역경도 {point['x']}")
                else:
                    # 지번/도로명 주소 모두 실패하면 아래 메시지 출력
                    print(f"주소 '{new_addr}' 좌표 변환 재시도 실패")

            else: # API 요청 자체가 실패한 경우 아래 메시지 출력
                print(f"주소 '{new_addr}' 요청 실패: 상태 코드 {response.status_code}")
    else:
        print(f"주소 '{new_addr}' 요청 실패: 상태 코드 {response.status_code}")
# 결과적으로 df1에는 수정 전 주소가 수정 후 주소로 바뀌고, 위도/경도도 정확하게 갱신됨

df1.to_csv('부산광역시 도시철도 동해선.csv')
# 수정한 최종 데이터프레임을 부산광역시 도시철도 동해선.csv에 덮어쓰기

### 출력 결과

주소 '부산광역시 기장군 일광면 이천리 750-4' → '부산광역시 기장군 일광읍 이천리 750-4' 변경 완료 및 좌표 수정 완료: 위도 35.26744938601336, 경도 129.23307958779256

주소 '울산시 남구 상개리 97-1' → '울산광역시 남구 상개동 97-1' 변경 완료 및 좌표 수정 완료: 위도 35.507860987546024, 경도 129.3212896620817